In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
from multiprocessing import Pool        # Multiprocess Runs
from tqdm import tqdm_notebook, tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from attention import attention_3d_block
from tensorflow.keras.models import Model

import seaborn as sns
import matplotlib.pyplot as plt

sys.path.insert(0, os.path.abspath('../'))
from module.prepare_data import *
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import KFold

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices: tf.config.experimental.set_memory_growth(device, True)

In [5]:
########################### Vars
#################################################################################
VER = 3                          # Our model version
SEED = 42
TARGET      = 'sales'            # Our target
START_TRAIN = 700                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913               # End day of our train set
P_HORIZON   = 28                 # Prediction horizon
#STORES ids
ORIGINAL = '../input/m5-forecasting-accuracy/'
STORES_IDS = pd.read_csv(ORIGINAL+'sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())
#PATHS for Features
ORIGINAL = '../input/m5-forecasting-accuracy/'
BASE     = '../cache/ori_grid_part_1.pkl'
PRICE    = '../cache/ori_grid_part_2.pkl'
CALENDAR = '../cache/ori_grid_part_3.pkl'
LAGS     = '../cache/ori_lags_df_28.pkl'
MEAN_ENC = '../cache/ori_mean_encoding_df.pkl'
BASE_PATH = '../cache'
FINAL_TARGETS = 'sales'
SAV_BASE_PATH = '../cache/ver3'
PKL_BASE_PATH = BASE_PATH

########################### caculate mean and std
#################################################################################
# diff_series = []
# for id_, group in tqdm(BASE_GRID_DF[['id','d','sales']].groupby('id')):
#     diff_series += group['sales'].diff().dropna().tolist()
# diff_mean = np.mean(diff_series)
# diff_std = np.std(diff_series)
diff_mean, diff_std = 0.00022844736211235283, 2.9674834203072016

In [6]:
########################### dataset sample
#################################################################################
"""
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(2).prefetch(1)
for x, y in dataset:
    print('x = ', x.numpy())
    print('y = ', y.numpy())
    
a  = gen_dataset(np.arange(100), window_size)
b = a.concatenate(gen_dataset(np.arange(2000,2100), window_size))
b = b.batch(batch_size).prefetch(1)


validaton_span = 28
window_size = 28
batch_size = 5
shuffle_buffer = 2000
train_dataset = train_dataset.shuffle(shuffle_buffer)
train_dataset = train_dataset.batch(5).prefetch(1)
valid_series = train_dataset.batch(1).prefetch(1)

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size+1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset
    
    
"""





"\ndataset = tf.data.Dataset.range(10)\ndataset = dataset.window(5, shift=1, drop_remainder=True)\ndataset = dataset.flat_map(lambda window: window.batch(5))\ndataset = dataset.map(lambda window: (window[:-1], window[-1:]))\ndataset = dataset.shuffle(buffer_size=10)\ndataset = dataset.batch(2).prefetch(1)\nfor x, y in dataset:\n    print('x = ', x.numpy())\n    print('y = ', y.numpy())\n    \na  = gen_dataset(np.arange(100), window_size)\nb = a.concatenate(gen_dataset(np.arange(2000,2100), window_size))\nb = b.batch(batch_size).prefetch(1)\n\n\nvalidaton_span = 28\nwindow_size = 28\nbatch_size = 5\nshuffle_buffer = 2000\ntrain_dataset = train_dataset.shuffle(shuffle_buffer)\ntrain_dataset = train_dataset.batch(5).prefetch(1)\nvalid_series = train_dataset.batch(1).prefetch(1)\n\ndef windowed_dataset(series, window_size, batch_size, shuffle_buffer):\n    dataset = tf.data.Dataset.from_tensor_slices(series)\n    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)\n    

In [7]:
# BASE_GRID_DF = load_base_features(PKL_BASE_PATH, SAV_BASE_PATH, FINAL_TARGETS)

# BASE_GRID_DF['item_id']= BASE_GRID_DF['item_id'].apply(lambda x: int(x.split('_')[-1]))
# BASE_GRID_DF['dept_id']= BASE_GRID_DF['dept_id'].apply(lambda x: int(x.split('_')[-1]))
# BASE_GRID_DF['cat_id']= BASE_GRID_DF['cat_id'].replace({'HOBBIES':0, 'HOUSEHOLD':1, 'FOODS':2})
# for col in ['event_name_1','event_type_1','event_name_2','event_type_2']:
#     BASE_GRID_DF[col] = BASE_GRID_DF[col].replace(dict(zip(BASE_GRID_DF[col].unique(), np.arange(BASE_GRID_DF[col].unique().shape[0]))))
# BASE_GRID_DF['snap_CA'] = BASE_GRID_DF['snap_CA'].astype(int)
# BASE_GRID_DF['snap_TX'] = BASE_GRID_DF['snap_TX'].astype(int)
# BASE_GRID_DF['snap_WI'] = BASE_GRID_DF['snap_WI'].astype(int)

# print(BASE_GRID_DF.info())

In [8]:
MLP_COLUMNS = ['item_id', 'dept_id', 'cat_id', 
       'release', 'sell_price', 'price_max', 'price_min', 'price_std',
       'price_mean', 'price_norm', 'price_nunique', 'item_nunique',
       'price_momentum', 'price_momentum_m', 'price_momentum_y',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 'tm_y',
       'tm_wm', 'tm_dw', 'tm_w_end', 'enc_store_id_cat_id_mean',
       'enc_store_id_cat_id_std', 'enc_store_id_dept_id_mean',
       'enc_store_id_dept_id_std', 'enc_store_id_item_id_mean',
       'enc_store_id_item_id_std', 'enc_store_id_tm_dw_item_id_mean',
       'enc_store_id_tm_dw_item_id_std', 'enc_store_id_tm_dw_mean',
       'enc_store_id_tm_dw_std']

In [9]:
def train_model2():
    """
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[None]),
            tf.keras.layers.Conv1D(filters=32,kernel_size=5,strides=1,padding="causal",activation="relu",input_shape=[None,1]),
            tf.keras.layers.LSTM(64, return_sequences=True),
            tf.keras.layers.LSTM(64),
            tf.keras.layers.Dense(1),
    #         tf.keras.layers.Lambda(lambda x: x*100),
        ]
    )
    optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum =0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=[tf.keras.metrics.RootMeanSquaredError()])
    model.fit(train_dataset, validation_data=valid_dataset, epochs=20, callbacks=[lr_schedule], verbose=1)
    """
    raise Exception('no implement')

In [10]:
# window_size = 28
def gen_dataset(series, window_size):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 28, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size+28))
    dataset = dataset.map(lambda window: (window[:-28], window[-28:]))
    return dataset

def get_train_valid(grid, window_size = 28):
    
    X_train, y_train = [],[]
    X_train2 = []
    for id_, group in grid.groupby('id'):
        if group.shape[0] == 0:
            continue
        train_dataset = gen_dataset(group['sales'], window_size)
        old_n_X_train = len(X_train)
        for idx, (x, y) in enumerate(train_dataset):
            X_train.append(x.numpy())# = np.vstack([X_train, x.numpy()])
            y_train.append(y.numpy())# = np.vstack([y_train, y.numpy()])
        
        if (len(X_train)-old_n_X_train) != (group.iloc[window_size:idx+1+window_size].shape[0]):
            print(id_, len(X_train)-old_n_X_train, group.iloc[window_size:idx+1+window_size].shape[0], group.shape[0])
        else:
            X_train2.append(group.iloc[window_size:idx+1+window_size][MLP_COLUMNS])
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_train2 = np.vstack(X_train2)
    
    return X_train, y_train, X_train2


def build_graph(n_timeseries, n_mlp_features, weights_path=None):
    input1 = tf.keras.layers.Input(shape=[n_timeseries])
    input2 = tf.keras.layers.Input(shape=[n_mlp_features])
    x1 = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[n_timeseries])(input1)
    x1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True))(x1)
    x1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64))(x1)
    x1 = tf.keras.layers.Lambda(lambda x: x*100)(x1)
    
    x2 = input2#tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[n_mlp_features])(input2)
    x = tf.keras.layers.concatenate([x1,x2])
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
#     x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    output = tf.keras.layers.Dense(28, activation='relu')(x)
#     output = tf.keras.layers.Lambda(lambda x: x*100)(x)

    model = Model(inputs=[input1, input2], outputs=[output])
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(
        lambda epoch: 1e-8 * 10 **(epoch/20)
    )
    optimizer = tf.keras.optimizers.SGD(lr=1e-6, momentum =0.9)
    model.compile(loss='mse', optimizer=optimizer, metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
    if type(None) != type(weights_path):
        model.load_weights(weights_path)

    return model


def train_model(X_train, X2_train, y_train, X_valid, X2_valid, y_valid, store_id, base_path, n_timeseries, n_mlp_features, batch_size=10, epochs=1):
    model = build_graph(n_timeseries, n_mlp_features, None)
    his = model.fit([X_train,X2_train], y_train, validation_data=([X_valid,X2_valid], y_valid), batch_size=batch_size, epochs=epochs, verbose=1)
    model.save_weights(f'{base_path}/bilstm_{store_id}.h5')
    return model

# def predict_series(model, series, mlp_input, n_timeseries):
    
#     lst = []
#     input1 = series
#     input2 = mlp_input
#     for i in range(29):
#         pred = model.predict([input1.reshape(1,n_timeseries), input2])
#         input1 = np.hstack((input1,pred[0]))[-n_timeseries:]
#         lst.append(pred[0][0])
#     return np.array(lst)

def predict_samples(grid_df, model):
    pred_df = pd.DataFrame(columns=['id']+[f'F{i}' for i in range(1,29)])
    idx = 0
    for id_, group in tqdm(grid_df.groupby('id')):
        if group.shape[0] == 0:
            continue
        input1 = group[group['d']<=1913]['sales'].values[-28:].reshape(1,28)
        input2 = group[group['d']<=1913].iloc[-1][MLP_COLUMNS].values.reshape(1,39)
        predictions = model.predict([input1.astype(float), input2.astype(float)])
        pred_df.loc[idx] = dict(zip(pred_df.columns.tolist(),[id_,*predictions[0]]))
        idx += 1
    return pred_df

def rmse(y, y_pred):
    return np.sqrt(np.mean(np.square(y - y_pred)))

In [11]:
# store_id = 'CA_1'
n_fold = 3
part_len = (END_TRAIN-START_TRAIN)//n_fold

In [12]:
STORE_IDS = pd.read_pickle(f'{SAV_BASE_PATH}/BASE_FEATURES.pkl').store_id.unique().tolist()

In [13]:
pred_df = pd.DataFrame()
for store_id in STORE_IDS:
    BASE_GRID_DF = pd.read_pickle(f'{SAV_BASE_PATH}/BASE_FEATURES.pkl')
    grid_df = BASE_GRID_DF[(BASE_GRID_DF['store_id']==store_id)]
    del BASE_GRID_DF
    part1 = (grid_df['d']>START_TRAIN)&(grid_df['d']<=END_TRAIN-2*part_len)
    part2 = (grid_df['d']>START_TRAIN+part_len)&(grid_df['d']<=END_TRAIN-part_len)
    part3 = (grid_df['d']>START_TRAIN+2*part_len)&(grid_df['d']<=END_TRAIN)

    X_part1, y_part1, X2_part1 = get_train_valid(grid_df[part1])
    X_part2, y_part2, X2_part2 = get_train_valid(grid_df[part2])
    X_part3, y_part3, X2_part3 = get_train_valid(grid_df[part3])
    
    np.save(f'{SAV_BASE_PATH}/X_part1_{store_id}.npy', X_part1) # save
    np.save(f'{SAV_BASE_PATH}/X2_part1_{store_id}.npy', X2_part1) # save
    np.save(f'{SAV_BASE_PATH}/y_part1_{store_id}.npy', y_part1) # save
    np.save(f'{SAV_BASE_PATH}/X_part2_{store_id}.npy', X_part2) # save
    np.save(f'{SAV_BASE_PATH}/X2_part2_{store_id}.npy', X2_part2) # save
    np.save(f'{SAV_BASE_PATH}/y_part2_{store_id}.npy', y_part2) # save
    np.save(f'{SAV_BASE_PATH}/X_part3_{store_id}.npy', X_part3) # save
    np.save(f'{SAV_BASE_PATH}/X2_part3_{store_id}.npy', X2_part3) # save
    np.save(f'{SAV_BASE_PATH}/y_part3_{store_id}.npy', y_part3) # save
    
    # X_part1 = np.load(f'X_part1.npy') # load
    # y_part1 = np.load(f'y_part1.npy') # load
    # X_part2 = np.load(f'X_part2.npy') # load
    # y_part2 = np.load(f'y_part2.npy') # load
    # X_part3 = np.load(f'X_part3.npy') # load
    # y_part3 = np.load(f'y_part3.npy') # load
    
    X_train = np.vstack([X_part1, X_part2])
    X2_train = np.vstack([X2_part1, X2_part2])
    y_train = np.vstack([y_part1, y_part2])
    X_valid = X_part3
    X2_valid = X2_part3
    y_valid = y_part3
    train_sample_index = np.random.choice(X_train.shape[0], 500_000)
    valid_sample_index = np.random.choice(X_valid.shape[0], 200_000)
    
    model = train_model(X_train[train_sample_index][:,:], X2_train[train_sample_index], y_train[train_sample_index], 
                    X_valid[valid_sample_index][:,:], X2_valid[valid_sample_index], y_valid[valid_sample_index], 
                    store_id, SAV_BASE_PATH, 28, 39, epochs=20)
    
    pred_df_i = predict_samples(grid_df, model)
    pred_df = pd.concat([pred_df, pred_df_i], axis=0)
#     break

FOODS_2_007_CA_1_validation 0 27 55
FOODS_2_031_CA_1_validation 0 13 41
FOODS_2_032_CA_1_validation 0 13 41
FOODS_2_035_CA_1_validation 0 6 34
FOODS_3_044_CA_1_validation 0 6 34
FOODS_3_105_CA_1_validation 0 6 34
FOODS_3_205_CA_1_validation 0 6 34
FOODS_3_229_CA_1_validation 0 20 48
FOODS_3_270_CA_1_validation 0 13 41
FOODS_3_275_CA_1_validation 0 6 34
FOODS_3_359_CA_1_validation 0 27 55
FOODS_3_429_CA_1_validation 0 13 41
FOODS_3_482_CA_1_validation 0 27 55
FOODS_3_521_CA_1_validation 0 6 34
FOODS_3_557_CA_1_validation 0 13 41
FOODS_3_693_CA_1_validation 0 20 48
HOBBIES_1_219_CA_1_validation 0 27 55
HOBBIES_1_354_CA_1_validation 0 6 34
HOUSEHOLD_1_283_CA_1_validation 0 13 41
HOUSEHOLD_1_332_CA_1_validation 0 6 34
HOUSEHOLD_1_384_CA_1_validation 0 6 34
HOUSEHOLD_2_114_CA_1_validation 0 20 48
HOUSEHOLD_2_132_CA_1_validation 0 6 34
HOUSEHOLD_2_307_CA_1_validation 0 6 34
HOUSEHOLD_2_360_CA_1_validation 0 6 34
HOUSEHOLD_2_487_CA_1_validation 0 27 55
HOUSEHOLD_2_515_CA_1_validation 0 6 34
F

100%|█████████▉| 30361/30490 [02:02<00:00, 251.93it/s]

FOODS_3_418_CA_2_validation 0 6 34
FOODS_3_482_CA_2_validation 0 6 34
FOODS_3_557_CA_2_validation 0 6 34
FOODS_3_693_CA_2_validation 0 20 48
FOODS_3_761_CA_2_validation 0 13 41
HOBBIES_1_219_CA_2_validation 0 27 55
HOBBIES_1_334_CA_2_validation 0 6 34
HOBBIES_1_339_CA_2_validation 0 6 34
HOBBIES_1_354_CA_2_validation 0 6 34
HOUSEHOLD_1_018_CA_2_validation 0 20 48
HOUSEHOLD_1_022_CA_2_validation 0 13 41
HOUSEHOLD_1_267_CA_2_validation 0 20 48
HOUSEHOLD_1_283_CA_2_validation 0 6 34
HOUSEHOLD_1_332_CA_2_validation 0 20 48
HOUSEHOLD_1_384_CA_2_validation 0 13 41
HOUSEHOLD_2_132_CA_2_validation 0 6 34
HOUSEHOLD_2_175_CA_2_validation 0 6 34
HOUSEHOLD_2_177_CA_2_validation 0 13 41
HOUSEHOLD_2_307_CA_2_validation 0 13 41
FOODS_2_305_CA_2_validation 0 18 46
FOODS_3_003_CA_2_validation 0 25 53
FOODS_3_047_CA_2_validation 0 25 53
FOODS_3_119_CA_2_validation 0 25 53
FOODS_3_134_CA_2_validation 0 4 32
FOODS_3_147_CA_2_validation 0 18 46
FOODS_3_166_CA_2_validation 0 25 53
FOODS_3_169_CA_2_validatio

100%|██████████| 30490/30490 [02:10<00:00, 234.10it/s]


FOODS_2_096_CA_3_validation 0 20 48
FOODS_3_044_CA_3_validation 0 6 34
FOODS_3_084_CA_3_validation 0 27 55
FOODS_3_105_CA_3_validation 0 6 34
FOODS_3_218_CA_3_validation 0 6 34
FOODS_3_229_CA_3_validation 0 6 34
FOODS_3_270_CA_3_validation 0 20 48
FOODS_3_275_CA_3_validation 0 27 55
FOODS_3_359_CA_3_validation 0 27 55
FOODS_3_429_CA_3_validation 0 13 41
FOODS_3_482_CA_3_validation 0 27 55
FOODS_3_525_CA_3_validation 0 13 41
HOBBIES_1_219_CA_3_validation 0 13 41
HOBBIES_1_354_CA_3_validation 0 6 34
HOBBIES_2_041_CA_3_validation 0 27 55
HOUSEHOLD_1_018_CA_3_validation 0 20 48
HOUSEHOLD_1_332_CA_3_validation 0 20 48
HOUSEHOLD_1_384_CA_3_validation 0 6 34
HOUSEHOLD_1_435_CA_3_validation 0 13 41
HOUSEHOLD_1_450_CA_3_validation 0 6 34
HOUSEHOLD_1_495_CA_3_validation 0 6 34
HOUSEHOLD_2_005_CA_3_validation 0 6 34
HOUSEHOLD_2_060_CA_3_validation 0 13 41
HOUSEHOLD_2_114_CA_3_validation 0 20 48
HOUSEHOLD_2_132_CA_3_validation 0 6 34
HOUSEHOLD_2_175_CA_3_validation 0 20 48
HOUSEHOLD_2_177_CA_3_val

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



500000/500000 [==============================] - 325s 650us/sample - loss: 7.6717 - root_mean_squared_error: 2.7698 - val_loss: 6.3947 - val_root_mean_squared_error: 2.5288
Epoch 19/20
169070/500000 [=========>....................] - ETA: 2:55 - loss: 7.2058 - root_mean_squared_error: 2.6844

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



500000/500000 [==============================] - 327s 655us/sample - loss: 7.6675 - root_mean_squared_error: 2.7690 - val_loss: 6.3966 - val_root_mean_squared_error: 2.5291


100%|██████████| 30490/30490 [02:17<00:00, 221.05it/s]


In [14]:
for col in [f'F{i}' for i in range(1,29)]:
    print(col, pred_df[col].sum())

F1 40586.18937710277
F2 29395.143401602167
F3 38120.38545977829
F4 36982.47288805284
F5 33591.79892578081
F6 39587.439021511425
F7 21188.550194994838
F8 30484.770024415222
F9 35838.44382213248
F10 42286.74087745728
F11 27302.81191282044
F12 31864.939928364183
F13 36296.47553243057
F14 31806.939007062843
F15 36411.397060816176
F16 40450.16693109856
F17 17131.708616930875
F18 33480.58410119888
F19 23496.48269185616
F20 34811.43517611466
F21 38752.0814780636
F22 39937.21767022612
F23 27844.90706532309
F24 40845.706697984686
F25 25612.66726968481
F26 39288.13686922821
F27 39248.542891609424
F28 34933.17951326945


In [16]:
pred_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.0,0.928475,1.088036,0.856354,1.026586,0.547378,0.0,0.0,1.222985,...,1.036828,0.810852,0.573717,0.759222,0.0,1.016626,0.0,0.601889,0.0,0.689259
1,FOODS_1_002_CA_1_validation,0.0,0.455638,0.636054,0.360710,0.294070,0.092625,0.0,0.0,0.830625,...,0.361501,0.190376,0.128430,0.463296,0.0,0.762140,0.0,0.177825,0.0,0.411012
2,FOODS_1_003_CA_1_validation,0.0,0.553006,0.494741,0.323444,0.643956,0.142067,0.0,0.0,0.695731,...,0.676567,0.619101,0.481467,0.353034,0.0,0.724214,0.0,0.561560,0.0,0.398130
3,FOODS_1_004_CA_1_validation,0.0,3.015525,2.188738,1.849086,2.785250,2.958735,0.0,0.0,5.067468,...,4.352099,4.261330,5.083395,5.753962,0.0,4.373944,0.0,3.045892,0.0,5.907110
4,FOODS_1_005_CA_1_validation,0.0,1.465038,1.265729,1.287864,1.192603,1.236912,0.0,0.0,1.745727,...,1.239331,1.134817,1.429517,1.915966,0.0,1.489210,0.0,1.213023,0.0,1.360599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
60976,FOODS_3_824_WI_3_evaluation,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
60977,FOODS_3_825_WI_3_evaluation,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
60978,FOODS_3_826_WI_3_evaluation,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [15]:
sample_submission = pd.read_csv(f'{ORIGINAL}/sample_submission.csv')
pred_df = pd.concat([pred_df, sample_submission[~sample_submission['id'].isin(pred_df.id.unique().tolist())]], axis=0)
pred_df.to_csv('../cache/submission/lstm_submission.csv', index=False)